In [1]:
# Starts the autoreload extension, which allows editing the .py files with the notebook running and automatically imports the latest changes

%load_ext autoreload
%autoreload 2

import trim_duplicates, model, network, gradcam, plots
from dataset import Dataset
import haiku as hk
import jax.numpy as jnp
import numpy as np
import jax
import sklearn
import wandb
from trim_duplicates import DuplicatesData
import matplotlib.pyplot as plt

assert jax.local_device_count() >= 8

NUM_CLASSES = 4
SEED = 12
BATCH_SIZE = 128

def basemodel_process(x): return x

In [3]:
rng = jax.random.PRNGKey(SEED)
dataset_mendeley = Dataset.load("data/mendeley", rng=rng, official_split=False)
dataset_tawsifur = Dataset.load("data/tawsifur", rng=rng, official_split=False)
print("Loaded mendeley", dataset_mendeley.classnames)
print("Loaded tawsifur", dataset_tawsifur.classnames)

FileNotFoundError: [Errno 2] No such file or directory: 'data/mendeley/x_train.npy'

In [ ]:
def show_diff(ds, global_set, max_rows=4):
    dups = DuplicatesData.load("dup_data/duprem_base_" + ds.name + "_CV_0.pickle")
    dups_pix = DuplicatesData.load("dup_data/duprem_base_" + ds.name + "_CV_0pix_.pickle")

    assert global_set == "embed" or global_set == "pix"

    if global_set == "embed":
        global_set = dups.indices
        other_set = dups_pix.indices
    else:
        global_set = dups_pix.indices
        other_set = dups.indices
    
    diff = global_set - dups.indices.intersection(dups_pix.indices)

    assert jnp.all(ds.rng == dups.rng) and jnp.all(ds.rng == dups_pix.rng)

    other_dups_map = {}
    for group in other_set:
        if len(group) > 1:
            for i in group:
                other_dups_map[i] = True

    imgs = []
    for v in diff:
        imgs.append([])
        if len(v) > 1:
            for i in v:
                imgs[len(imgs)-1].append({ "img": ds.x_all[i,:,:,:], "color": "red" if i not in other_dups_map else "black" })
    
    plots.compare_n_images(imgs, rows=max_rows)

show_diff(dataset_mendeley, "embed")

NameError: name 'dataset_mendeley' is not defined

In [ ]:
show_diff(dataset_mendeley, "pix")

In [ ]:
dataset_mendeley.x_all.shape